# Install Necessary Libraries

In [ ]:
%%capture
!pip install translators
!pip install git+https://github.com/csebuetnlp/normalizer

# Import Libraries

In [ ]:
import re
import pandas as pd
from normalizer import normalize
from tqdm.auto import tqdm
import translators as ts

Using region Nevada server backend.



In [ ]:
tqdm.pandas()

# Load the Dataset

In [ ]:
dataset_file_location = '/content/drive/MyDrive/B.Tech. Final Year Project/Offensive Language Detection/Datasets/m_dataset_21_9/merged_dataset_21_9_arnab.csv'
df = pd.read_csv(dataset_file_location)
df.count()

text    90500
hate    90501
dtype: int64

In [ ]:
df['hate'].value_counts()

0    52089
1    38412
Name: hate, dtype: int64

# Data Pre-processing

## Drop rows with missing values in the text column

In [ ]:
df = df.dropna(subset=["text"])

## Remove Censored Comments
Some comments have multiple stars(```*```) to hide abusive words. Those comments are removed.

In [ ]:
df_clean = df[~df['text'].str.contains(r'\*{2,}', regex=True)].copy()

## Remove Special Words

In [ ]:
special_words = ['<br />', ':D', ':P', ':v']

def remove_special_wrods(string):
    for word in special_words:
        string = string.replace(word, ' ')
    return string

In [ ]:
df_clean['text'] = df_clean['text'].progress_apply(remove_special_wrods)

  0%|          | 0/90392 [00:00<?, ?it/s]

## Remove Links and Emojis
[Normalizer](https://github.com/csebuetnlp/normalizer) is used.

In [ ]:
df_clean['text'] = df_clean['text'].progress_apply(lambda x: normalize(x, url_replacement='', emoji_replacement=''))

  0%|          | 0/90392 [00:00<?, ?it/s]

## Remove Single-Letter Words

In [ ]:
df_clean['text'] = df_clean['text'].progress_apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

  0%|          | 0/90392 [00:00<?, ?it/s]

## Translate English words to Bengali

If the below code gives an error like:<br />
**```KeyError: 0```**<br />

Then run the code until and unless the error is gone.<br />
This can happen due to some problem in the translator.


In [ ]:
iter = 0
num_of_eng_words = {0: 0}

In [ ]:
while True:
    iter += 1
    count = 0
    print("Iteration:", iter)
    for row in tqdm(df_clean.itertuples(), total=df_clean.count()['text'], desc='Translating English Words'):
        temp_text = row.text
        for word in temp_text.split():
            if re.match(r'^[a-zA-Z]+$', word):
                count += 1
                word_to_bn = ts.translate_text(word, translator='google', from_language='en', to_language='bn')
                if word_to_bn == word:
                    word_to_bn = ''
                temp_text = temp_text.replace(word, word_to_bn)
                df_clean.loc[row.Index, 'text'] = temp_text
    if num_of_eng_words[iter - 1] == count:
        print(count, " sentences can not be translated.")
        break
    else:
        num_of_eng_words[iter] = count

if 0 in num_of_eng_words.keys():
    del num_of_eng_words[0]
print("Number of English Words found in the Dataset:")
for iter, count in num_of_eng_words.items():
    print(f'Interation {iter}: {count}')

Iteration: 2


Translating English Words:   0%|          | 0/90392 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
for row in tqdm(df_clean.itertuples(), total=df_clean.count()['text'], desc='Checking for English Words'):
    temp_text = row.text
    for word in temp_text.split():
        if re.match(r'^[a-zA-Z]+$', word):
            temp_text = temp_text.replace(word, '')
            df_clean.loc[row.Index, 'text'] = temp_text
            print(row.Index, word)

Checking for English Words:   0%|          | 0/90348 [00:00<?, ?it/s]

## Strip any Whitespace Characters from the ```text``` Column

In [ ]:
df_clean["text"] = df_clean["text"].str.strip()

## Remove Blank Comments

In [ ]:
df_clean.drop(df_clean[df_clean['text'].str.len() == 0].index, axis=0, inplace=True)
df_clean.reset_index(drop=True, inplace=True)

# Cleaned Dataset Details

In [ ]:
df_clean.count()

text    90348
hate    90348
dtype: int64

In [ ]:
print(df.count()['text'] - df_clean.count()['text'], "rows were removed after pre-processing.")

152 rows were removed after pre-processing.


In [ ]:
print("Maximum string length:", df_clean['text'].str.len().max())
print("Minimum string length:", df_clean['text'].str.len().min())

Maximum string length: 3463
Minimum string length: 2


In [ ]:
print("Maximum number of words in a sentence :", df_clean['text'].str.split().str.len().max())
print("Minimum number of words in a sentence :", df_clean['text'].str.split().str.len().min())

Maximum number of words in a sentence : 572
Minimum number of words in a sentence : 1


# Saving the Cleaned Dataset

In [ ]:
df_clean_name = 'cleaned_12_10_'+dataset_file_location.rsplit('/', 1)[-1].rsplit('.')[0]
df_folder_name = dataset_file_location.rsplit('/', 1)[0]
df_clean.to_csv(df_folder_name+'/'+df_clean_name+'.csv', index=False)